In [1]:
import pandas as pd
import numpy as np

from itertools import product

import torch
from torch.utils.data import DataLoader

from tqdm.auto import tqdm

from models.Wd_Xlm_T import Wd_Xlm_T
from models.TweetDataset import TweetDataset

import transformers 

from utils.utils import (load_test_data, target_features, numerical_features,
                         categorical_features, features, MAX_LEN, create_dataset, get_results_df)

In [2]:
DATA_PATH = "./data/"
CHECKPOINT_PATH = "./checkpoints/"

BATCH_SIZE = 32

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
test_df = load_test_data(DATA_PATH)
ground_truths = test_df.loc[:, target_features].values

In [4]:
config = transformers.XLMRobertaConfig().from_pretrained(CHECKPOINT_PATH+"epoch_0_end")
model = Wd_Xlm_T.from_pretrained(CHECKPOINT_PATH+"epoch_0_end",  config = config,
                                            dim_features=len(numerical_features + features),
                                 dim_hidden=[768,512,256,128,64,32])

model = model.to(DEVICE)

In [5]:
tokenizer = transformers.XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

dataset = create_dataset(test_df, tokenizer)

In [6]:
model.eval()

predictions = []

dataloader = DataLoader(dataset, batch_size = BATCH_SIZE,
                        shuffle=False, drop_last=False)

for data in tqdm(dataloader):
    input_ids = data['input_ids'].to(DEVICE)
    attention_mask = data['attention_mask'].to(DEVICE)
    features = data['features'].to(DEVICE)

    labels = data['labels'].to(DEVICE)
        
    logits = model(input_ids, attention_mask, features)
    predictions.append(torch.sigmoid(logits).detach().cpu())

  0%|          | 0/43523 [00:00<?, ?it/s]

In [7]:
predictions = [prediction.numpy() for prediction in predictions]

In [8]:
prediction_arr = np.concatenate(predictions, axis=0)
print(prediction_arr.shape)

(1392727, 4)


In [9]:
results_df = get_results_df(prediction_arr, ground_truths)

results_df

reply    retweet  retweet_comment       like
rce       17.537952  21.993606         6.028145  16.199487
avg_prec   0.213977   0.464510         0.053691   0.706662